In [1]:
import tensorflow as tf

import tensorflow.keras as keras
import tensorflow.keras.utils as utils
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
import ta 
import datetime

In [2]:

csvFileAddress = 'csv\FTSE100_M15_202101210130_202206271215.csv'

DJ = pd.read_csv(csvFileAddress,delim_whitespace=True)

DJ['<ISGREEN>'] =  DJ['<CLOSE>'] > DJ['<OPEN>']
DJ['<SIZE>'] =  (DJ['<CLOSE>'] - DJ['<OPEN>']) / DJ['<CLOSE>'] *100
DJ['<VOLATILITY>'] =  DJ['<HIGH>'] - DJ['<LOW>'] /DJ['<CLOSE>']

DJ.drop(['<VOL>'],axis=1,inplace=True)

DJ.fillna(0,inplace=True)

DJ['<EMA30>']= ta.trend.ema_indicator( DJ['<CLOSE>'],window=30)
DJ['<EMA50>']= ta.trend.ema_indicator( DJ['<CLOSE>'],window=50)
DJ['<EMA200>']= ta.trend.ema_indicator( DJ['<CLOSE>'],window=200)

def Upper(e):
    if e['<ISGREEN>'] : 
        return e['<HIGH>']-e['<CLOSE>']
    return e['<HIGH>']-e['<OPEN>']

def Lower(e):
    if e['<ISGREEN>'] : 
        return e['<OPEN>']-e['<LOW>']
    return e['<CLOSE>']-e['<LOW>']


DJ['<UPPER>'] = DJ.apply(Upper,axis=1)
DJ['<LOWER>'] = DJ.apply(Lower,axis=1)


In [3]:

# Adding TA indicators : 

from ta.volatility import BollingerBands,KeltnerChannel,average_true_range
from ta.trend import MACD

DJ['<RSI>'] = ta.momentum.rsi(DJ['<CLOSE>'],window=15,fillna=1) / 100
# DJ['<CCI>'] = ta.trend.cci(close=DJ['<CLOSE>'],high=DJ['<HIGH>'],low=DJ['<LOW>'],window=14,fillna=1)


keltner = KeltnerChannel(close=DJ['<CLOSE>'],high=DJ['<HIGH>'],low=DJ['<LOW>'],window=50,window_atr=20,multiplier=3,fillna=1)
DJ['<KELTNER_H>'] = keltner.keltner_channel_hband()
DJ['<KELTNER_L>'] = keltner.keltner_channel_lband()
DJ['<KELTNER_M>'] = keltner.keltner_channel_mband()

DJ['<KELT_L_IND>'] = keltner.keltner_channel_lband_indicator()
DJ['<KELT_H_IND>'] = keltner.keltner_channel_hband_indicator()



bollinger =BollingerBands(close=DJ['<CLOSE>'],window=50,window_dev=3,fillna=True)
DJ['<BOL_H_IND>']=bollinger.bollinger_hband_indicator()
DJ['<BOL_L_IND>']=bollinger.bollinger_lband_indicator()

# macd = MACD(DJ['<CLOSE>'],fillna=True)


DJ['<ATR_24>'] = average_true_range(close=DJ['<CLOSE>'],high=DJ['<HIGH>'],low=DJ['<LOW>'],window=30).apply(lambda e : e/100)
DJ['<ATR_24_MULT>'] =abs(DJ['<SIZE>']) / DJ['<ATR_24>'] 


DJ["<ISGREEN>"] = DJ["<ISGREEN>"].astype(int)



In [4]:
timeFrame = (int(DJ.iloc[1][1][1])-int(DJ.iloc[0][1][1]) ) * 60 + (int(DJ.iloc[1][1][3:5])-int(DJ.iloc[0][1][3:5]))
CandlesInDay = 24 * (60//timeFrame)
timeFrame
def candleToTime(j):
    minuteMult = CandlesInDay//24
    k=j//minuteMult
    sth=timeFrame*(j%minuteMult)
    return '{:02d}:{:02d}:00'.format(k,sth)

uniqueDays = DJ.drop_duplicates(subset='<DATE>')
uniqueDays = pd.DataFrame(uniqueDays)

uniqueDaysCount=uniqueDays.shape[0]

newnumparr = np.full((uniqueDaysCount*CandlesInDay,2),'',dtype=np.object_)


for i in range(uniqueDaysCount):
    for j in range(CandlesInDay):
        newnumparr[(i*CandlesInDay)+j]=[uniqueDays.iloc[i][0],candleToTime(j)]

newDF = pd.DataFrame(newnumparr,columns=['<DATE>','<TIME>'])

newestDF = newDF.merge(DJ,on=['<DATE>','<TIME>'],how='left')


In [5]:
newestDF.drop(range(CandlesInDay),inplace=True)
newestDF.fillna(0.0,inplace=True)

In [6]:
newestDF.describe()

,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<TICKVOL>,<SPREAD>,<ISGREEN>,<SIZE>,<VOLATILITY>,<EMA30>,...,<RSI>,<KELTNER_H>,<KELTNER_L>,<KELTNER_M>,<KELT_L_IND>,<KELT_H_IND>,<BOL_H_IND>,<BOL_L_IND>,<ATR_24>,<ATR_24_MULT>
count,34464.000000,34464.000000,34464.000000,34464.000000,34464.000000,34464.000000,34464.000000,34464.000000,34464.000000,34464.000000,...,34464.000000,34464.000000,34464.000000,34464.000000,34464.000000,34464.000000,34464.000000,34464.000000,34464.000000,34464.000000
mean,6378.843759,6383.062451,6374.529065,6378.862787,414.946176,8.388986,0.443680,0.000219,6382.171895,6378.638973,...,0.452760,6386.971765,6369.910793,6378.441279,0.275592,0.343141,0.006906,0.009575,0.086448,0.629485
std,2245.848680,2247.318626,2244.345812,2245.855599,405.587873,21.762404,0.496825,0.096075,2247.009693,2245.717827,...,0.190603,2248.633274,2242.676935,2245.651743,0.446819,0.474765,0.082815,0.097385,0.048570,0.672775
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.020394,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6890.600000,6894.675000,6886.400000,6890.875000,135.000000,1.000000,0.000000,-0.037516,6893.676930,6892.021895,...,0.393835,6899.108667,6883.483167,6891.719167,0.000000,0.000000,0.000000,0.000000,0.062664,0.145783
50%,7111.600000,7115.550000,7106.800000,7111.500000,297.000000,2.000000,0.000000,0.000000,7114.550626,7111.243270,...,0.494420,7118.753667,7101.471000,7110.366333,0.000000,0.000000,0.000000,0.000000,0.082196,0.446326
75%,7340.600000,7345.300000,7335.125000,7340.600000,570.000000,4.000000,1.000000,0.039900,7344.300136,7343.345985,...,0.576572,7351.176167,7334.188000,7343.813000,1.000000,1.000000,0.000000,0.000000,0.109155,0.894164
max,7692.700000,7694.700000,7684.200000,7692.700000,3724.000000,105.000000,1.000000,1.382513,7693.701300,7672.180191,...,0.912642,7676.910667,7654.264000,7665.502667,1.000000,1.000000,1.000000,1.000000,0.340014,8.800167


In [7]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [27]:

import multiprocessing
import mp



df = newestDF.drop(['<DATE>','<TIME>'],axis=1)
threadCount = 12

trainNps=[]
labelValues = []

for i in range(threadCount): trainNps.append([])
for i in range(threadCount): labelValues.append([])

maxValues = df.max()

lenDFthread =  len(df) / threadCount
threads = []
p = multiprocessing.Pool(threadCount)
out=p.starmap(mp.thread_function,[(df.iloc[int(index*lenDFthread):int((1+index)*lenDFthread-1)],index,CandlesInDay) for index in range(threadCount)])



In [28]:
np.array(out,dtype=np.object_).shape

(12, 2, 1363)

In [29]:
datas=[]
labs=[]
for i in range(threadCount): 
    datas = datas+out[i][0]
    labs = labs + out[i][1]


In [30]:
len(datas),len(labs)

(16356, 16356)

In [31]:

l = len(datas)
windows2= np.array(datas)

labs = np.array(labs)


train_data = windows2[:int(l*.75)]
val_data = windows2[int(l*.75)+1:]



train_labels = labs[:int(l*.75)].astype(np.uint8)
val_labels = labs[int(l*.75)+1:].astype(np.uint8)


len(train_data),len(train_labels)


(12267, 12267)

In [32]:
len(val_data),len(val_labels)

(4088, 4088)

In [33]:
pd.DataFrame(labs,columns=['signal']).value_counts()/len(labs)

signal
0         0.406823
2         0.309122
1         0.284055
dtype: float64

In [ ]:
tf.random.set_seed(42)

from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D,LSTM, ConvLSTM1D,Flatten,SimpleRNN,GRU,AveragePooling2D

tf.random.set_seed(42)

model_rnn = keras.Sequential([
    LSTM(32,input_shape=train_data[0].shape,dropout=.1,return_sequences=1),
    LSTM(64,dropout=.1,return_sequences=1),
    LSTM(128,dropout=.1,return_sequences=0),
    # Flatten(),
    Dense(100,activation='relu'),
    Dense(3,activation='sigmoid'),

])

model_rnn.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(),
    optimizer = 'adam',
    metrics=['accuracy']
)

model_rnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 12, 32)            4992      
                                                                 
 lstm_1 (LSTM)               (None, 12, 64)            24832     
                                                                 
 lstm_2 (LSTM)               (None, 128)               98816     
                                                                 
 dense (Dense)               (None, 100)               12900     
                                                                 
 dense_1 (Dense)             (None, 3)                 303       
                                                                 
Total params: 141,843
Trainable params: 141,843
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_rnn.fit(train_data,train_labels,validation_data=(val_data,val_labels),batch_size=64,epochs=200,verbose=2)

Epoch 1/200
1/1 - 6s - loss: 1.0949 - accuracy: 0.3333 - val_loss: 1.0861 - val_accuracy: 0.5000 - 6s/epoch - 6s/step
Epoch 2/200
1/1 - 0s - loss: 1.0740 - accuracy: 0.5556 - val_loss: 1.0759 - val_accuracy: 0.5000 - 38ms/epoch - 38ms/step
Epoch 3/200
1/1 - 0s - loss: 1.0586 - accuracy: 0.5556 - val_loss: 1.0666 - val_accuracy: 0.5000 - 37ms/epoch - 37ms/step
Epoch 4/200
1/1 - 0s - loss: 1.0392 - accuracy: 0.5556 - val_loss: 1.0597 - val_accuracy: 0.5000 - 37ms/epoch - 37ms/step
Epoch 5/200
1/1 - 0s - loss: 1.0172 - accuracy: 0.5556 - val_loss: 1.0543 - val_accuracy: 0.5000 - 37ms/epoch - 37ms/step
Epoch 6/200
1/1 - 0s - loss: 0.9902 - accuracy: 0.5556 - val_loss: 1.0498 - val_accuracy: 0.5000 - 37ms/epoch - 37ms/step
Epoch 7/200
1/1 - 0s - loss: 0.9593 - accuracy: 0.5556 - val_loss: 1.0476 - val_accuracy: 0.5000 - 38ms/epoch - 38ms/step
Epoch 8/200
1/1 - 0s - loss: 0.9302 - accuracy: 0.5556 - val_loss: 1.0502 - val_accuracy: 0.5000 - 37ms/epoch - 37ms/step
Epoch 9/200
1/1 - 0s - loss: